In [31]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [32]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,noumea,NC,22-10-29 00:02:12,-22.2763,166.4572,84.18,71,9,7.05
1,1,taos,US,22-10-29 00:04:02,36.4072,-105.5731,54.00,26,0,0.00
2,2,paamiut,GL,22-10-29 00:02:59,61.9940,-49.6678,33.93,85,100,23.13
3,3,houma,US,22-10-28 23:59:48,29.5958,-90.7195,68.00,93,0,11.50
4,4,ponta do sol,PT,22-10-29 00:04:02,32.6667,-17.1000,72.01,82,16,5.53


In [33]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [34]:
# Create a Maximum Temperature Heatmap
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [35]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [36]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [40]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,noumea,NC,22-10-29 00:02:12,-22.2763,166.4572,84.18,71,9,7.05
5,5,abeche,TD,22-10-29 00:09:13,13.8292,20.8324,78.31,18,87,8.34
15,15,honavar,IN,22-10-29 00:11:53,14.2833,74.4500,77.14,69,86,4.09
18,18,atuona,PF,22-10-29 00:03:05,-9.8000,-139.0333,76.80,75,10,18.43
22,22,vaitape,PF,22-10-29 00:03:07,-16.5167,-151.7500,78.75,78,23,8.84
28,28,sao joao da barra,BR,22-10-29 00:03:10,-21.6403,-41.0511,75.27,87,82,8.12
30,30,alice springs,AU,22-10-29 00:03:11,-23.7000,133.8833,88.21,40,40,9.22
33,33,sao raimundo nonato,BR,22-10-29 00:03:12,-9.0153,-42.6994,81.10,53,52,3.98
34,34,hilo,US,22-10-29 00:01:49,19.7297,-155.0900,84.25,78,5,3.00
40,40,kapaa,US,22-10-29 00:00:50,22.0752,-159.3190,87.78,70,7,7.00


In [41]:
preferred_cities_df.count()

City_ID       137
City          137
Country       137
Date          137
Lat           137
Lng           137
Max Temp      137
Humidity      137
Cloudiness    137
Wind Speed    137
dtype: int64

In [42]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,noumea,NC,84.18,-22.2763,166.4572,
5,abeche,TD,78.31,13.8292,20.8324,
15,honavar,IN,77.14,14.2833,74.4500,
18,atuona,PF,76.80,-9.8000,-139.0333,
22,vaitape,PF,78.75,-16.5167,-151.7500,
28,sao joao da barra,BR,75.27,-21.6403,-41.0511,
30,alice springs,AU,88.21,-23.7000,133.8833,
33,sao raimundo nonato,BR,81.10,-9.0153,-42.6994,
34,hilo,US,84.25,19.7297,-155.0900,
40,kapaa,US,87.78,22.0752,-159.3190,


In [44]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [49]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [55]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [56]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))